In [1]:
import gentype

In [2]:
gentype.ping()

True

In [3]:
es = gentype.Ensembl()

In [4]:
samples = es.get_samples()

In [5]:
pops = es.get_populations()

In [37]:
# For all samples this will take time may multiprocess for multiple parallel requests
variants = es.get_variants(3,17671934, 17681934, samples)

Fetched 10 Variants
Fetched 20 Variants
Fetched 30 Variants
Fetched 40 Variants
Fetched 50 Variants
Fetched 60 Variants
Fetched 70 Variants
Fetched 80 Variants
Fetched 90 Variants
Fetched 100 Variants
Fetched 110 Variants
Fetched 120 Variants
Fetched 130 Variants
Fetched 140 Variants
Fetched 150 Variants
Fetched 160 Variants
Fetched 170 Variants
Fetched 180 Variants
Fetched 190 Variants
Fetched 200 Variants
Fetched 210 Variants
Fetched 220 Variants
Fetched 230 Variants
Fetched 240 Variants
Fetched 250 Variants
Fetched 260 Variants
Fetched 270 Variants
Fetched 280 Variants
Fetched 290 Variants
Fetched 300 Variants
Fetched 310 Variants
Fetched 320 Variants
Fetched 330 Variants
Fetched 340 Variants


In [43]:
es.save("variants")

In [44]:
es = gentype.Ensembl.load("variants")

In [45]:
individuals = list(map(lambda x: gentype.Individual(x, "X", variants), samples))

In [47]:
X_matrix = gentype.generate_X_matrix(individuals)

In [49]:
X_matrix.shape

(2504, 2, 334)

In [0]:
X_matrix[0]

In [48]:
import pickle

with open("X_matrix", "wb") as file1:
    pickle.dump(X_matrix, file1)